# Import the library

In [1]:
import os

import torch
from trainer import Trainer, TrainerArgs

# from TTS.bin.compute_embeddings import compute_embeddings
from compute_embeddings import compute_embeddings # use custom formatter without forking the lib
from TTS.bin.resample import resample_files
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import CharactersConfig, Vits, VitsArgs, VitsAudioConfig

from TTS.tts.utils.languages import LanguageManager
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

# from TTS.utils.downloaders import download_vctk
# from TTS.config import load_config
# from TTS.config.shared_configs import BaseDatasetConfig
# from TTS.tts.datasets import load_tts_samples
# from TTS.tts.utils.managers import save_file
# from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.datasets.formatters import vctk
from functools import partial

from tqdm import tqdm

torch.set_num_threads(24)

# Setup constants

In [2]:
# Current path
CURRENT_PATH = os.getcwd()

# Name of the run for the Trainer
RUN_NAME = "KhongKhunTTS-TH-TSync2"

# Path where you want to save the models outputs (configs, checkpoints and tensorboard logs)
OUT_PATH = os.path.join(CURRENT_PATH, "runs")

# If you want to do transfer learning and speedup your training you can set here the path to the model
RESTORE_PATH = "./best_model_YourTTS_VCTK.pth"

# This paramter is useful to debug, it skips the training epochs and just do the evaluation  and produce the test sentences
SKIP_TRAIN_EPOCH = False

# Set here the batch size to be used in training and evaluation
BATCH_SIZE = 32

# Training Sampling rate and the target sampling rate for resampling the downloaded dataset (Note: If you change this you might need to redownload the dataset !!)
# Note: If you add new datasets, please make sure that the dataset sampling rate and this parameter are matching, otherwise resample your audios
SAMPLE_RATE = 16000

# Max audio length in seconds to be used in training (every audio bigger than it will be ignored)
MAX_AUDIO_LEN_IN_SECONDS = 10

### Download VCTK dataset
VCTK_DOWNLOAD_PATH = os.path.join(CURRENT_PATH, "TSync2-to-vctk")
# Define the number of threads used during the audio resampling
NUM_RESAMPLE_THREADS = 10

# Dataset configuration

In [3]:
# init configs
tsync2_config = BaseDatasetConfig(
    formatter="vctk_16k",
    dataset_name="tsync2",
    meta_file_train="",
    meta_file_val="",
    path=VCTK_DOWNLOAD_PATH,
    language="th",
    ignored_speakers=[
        # "cv017", # Female Teenager
        # "cv048", # Female Teenager
        # "cv039", # Female Adult
        # "cv052", # Female Adult
        # "cv069", # Male Teenager
        # "cv054", # Male Teenager
        # "cv049", # Male Adult
        # "cv026", # Male Adult
    ], # For testing set
)

# Add here all datasets configs, in our case we just want to train with the VCTK dataset then we need to add just VCTK. Note: If you want to add new datasets, just add them here and it will automatically compute the speaker embeddings (d-vectors) for this new dataset :)
DATASETS_CONFIG_LIST = [tsync2_config]

# Extract speaker embeddings

In [4]:
SPEAKER_ENCODER_CHECKPOINT_PATH = (
    "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar"
)
SPEAKER_ENCODER_CONFIG_PATH = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json"

D_VECTOR_FILES = []  # List of speaker embeddings/d-vectors to be used during the training

vctk_16k = partial(
    vctk,
    wavs_path="wav16_silence_trimmed",
)

# Iterates all the dataset configs checking if the speakers embeddings are already computated, if not compute it
for dataset_conf in DATASETS_CONFIG_LIST:
    # Check if the embeddings weren't already computed, if not compute it
    embeddings_file = os.path.join(tsync2_config.path, "speakers.pth")
    if not os.path.isfile(embeddings_file):
        print(f">>> Computing the speaker embeddings for the {tsync2_config.dataset_name} dataset")
        compute_embeddings(
            SPEAKER_ENCODER_CHECKPOINT_PATH,
            SPEAKER_ENCODER_CONFIG_PATH,
            embeddings_file,
            formatter_name=tsync2_config.formatter,
            formatter=vctk_16k if tsync2_config.formatter == "vctk_16k" else None,
            dataset_name=tsync2_config.dataset_name,
            dataset_path=tsync2_config.path,
            meta_file_train=tsync2_config.meta_file_train,
            meta_file_val=tsync2_config.meta_file_val,
        )

    D_VECTOR_FILES.append(embeddings_file)

# Audio config used in training.

In [5]:
audio_config = VitsAudioConfig(
    sample_rate=SAMPLE_RATE,
    hop_length=256,
    win_length=1024,
    fft_size=1024,
    mel_fmin=0.0,
    mel_fmax=None,
    num_mels=80,
)

# Model configuration

In [6]:
# Init VITSArgs setting the arguments that are needed for the KhongKhunTTS model
model_args = VitsArgs(
    d_vector_file=D_VECTOR_FILES,
    use_d_vector_file=True,
    d_vector_dim=512,
    num_layers_text_encoder=10,
    speaker_encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    speaker_encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH,
    resblock_type_decoder="2",  # In the YourTTS paper, trained using ResNet blocks type 2, if you like you can use the ResNet blocks type 1 like the VITS model
    # Useful parameters to enable the Speaker Consistency Loss (SCL) described in the paper
    # use_speaker_encoder_as_loss=True,
    # Useful parameters to enable multilingual training
    use_language_embedding=True,
    embedded_language_dim=4,
)

In [7]:
# General training config, here you can change the batch size and others useful parameters
config = VitsConfig(
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name="KhongKhunTTS",
    run_description="""
            - KhongKhunTTS trained using TSync2 (VCTK structure)
        """,
    dashboard_logger="tensorboard",
    logger_uri=None,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=48,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=8,
    eval_split_max_size=256,
    print_step=50,
    plot_step=100,
    log_model_step=1000,
    save_step=5000,
    save_n_checkpoints=2,
    save_checkpoints=True,
    target_loss="loss_1",
    print_eval=False,
    use_phonemes=False,
    phonemizer="espeak",
    phoneme_language="en",
    compute_input_seq_cache=True,
    add_blank=True,
    text_cleaner="multilingual_cleaners",
    characters=CharactersConfig(
        characters_class="TTS.tts.models.vits.VitsCharacters",
        pad="_",
        eos="&",
        bos="*",
        blank=None,
        characters="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz\u00af\u00b7\u00df\u00e0\u00e1\u00e2\u00e3\u00e4\u00e6\u00e7\u00e8\u00e9\u00ea\u00eb\u00ec\u00ed\u00ee\u00ef\u00f1\u00f2\u00f3\u00f4\u00f5\u00f6\u00f9\u00fa\u00fb\u00fc\u00ff\u0101\u0105\u0107\u0113\u0119\u011b\u012b\u0131\u0142\u0144\u014d\u0151\u0153\u015b\u016b\u0171\u017a\u017c\u01ce\u01d0\u01d2\u01d4\u0430\u0431\u0432\u0433\u0434\u0435\u0436\u0437\u0438\u0439\u043a\u043b\u043c\u043d\u043e\u043f\u0440\u0441\u0442\u0443\u0444\u0445\u0446\u0447\u0448\u0449\u044a\u044b\u044c\u044d\u044e\u044f\u0451\u0454\u0456\u0457\u0491\u2013!\"'(),-.:;?|~ \u0e01\u0e02\u0e04\u0e06\u0e07\u0e08\u0e09\u0e0a\u0e0b\u0e0c\u0e0d\u0e0e\u0e0f\u0e10\u0e11\u0e12\u0e13\u0e14\u0e15\u0e16\u0e17\u0e18\u0e19\u0e1a\u0e1b\u0e1c\u0e1d\u0e1e\u0e1f\u0e20\u0e21\u0e22\u0e23\u0e24\u0e25\u0e27\u0e28\u0e29\u0e2a\u0e2b\u0e2c\u0e2d\u0e2e\u0e2f\u0e30\u0e31\u0e32\u0e33\u0e34\u0e35\u0e36\u0e37\u0e38\u0e39\u0e40\u0e41\u0e42\u0e43\u0e44\u0e45\u0e46\u0e47\u0e48\u0e49\u0e4a\u0e4b\u0e4c\u0e4d\u2014\u2018\u2019\u201c\u201d",
        punctuations="!\"'(),-.:;?|~ ",
        phonemes="",
        is_unique=True,
        is_sorted=True,
    ),
    phoneme_cache_path=None,
    precompute_num_workers=12,
    start_by_longest=True,
    datasets=DATASETS_CONFIG_LIST,
    cudnn_benchmark=False,
    max_audio_len=SAMPLE_RATE * MAX_AUDIO_LEN_IN_SECONDS,
    mixed_precision=False,
    test_sentences=[
        [
            "ทดสอบการอ่านออกเสียงภาษาไทย",
            "VCTK_TSync2",
            None,
            "th",
        ],
        [
            "ยายกินลำไยน้ำลายยายไหลย้อย",
            "VCTK_TSync2",
            None,
            "th",
        ],
        [
            "วันนี้ฉันจะไปกินข้าวมันไก่",
            "VCTK_TSync2",
            None,
            "th",
        ],
    ],
    # # Enable the weighted sampler
    # use_weighted_sampler=True,
    # # Ensures that all speakers are seen in the training batch equally no matter how many samples each speaker has
    # weighted_sampler_attrs={"speaker_name": 1.0},
    # weighted_sampler_multipliers={},
    # weighted_sampler_multipliers={"Makeitnotblack": None},

    # It defines the Speaker Consistency Loss (SCL) α to 9 like the paper
    speaker_encoder_loss_alpha=9.0,
)

In [8]:
# force the convertion of the custom characters to a config attribute
config.from_dict(config.to_dict())
config.weighted_sampler_multipliers = {}

# Training

In [9]:
# Load all the datasets samples and split traning and evaluation sets
train_samples, eval_samples = load_tts_samples(
    config.datasets,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
    formatter=vctk_16k if tsync2_config.formatter == "vctk_16k" else None,
)

 | > Found 2710 files in /home/titor/Capstone/dubbing-ai/KhongKhunTTS/TSync2-to-vctk


In [10]:
# init audio processor
# audio_config = config.audio.to_dict()
# audio_config['do_rms_norm'] = True
# audio_config['db_level'] = -27.0
# ap = AudioProcessor(**audio_config)

# config.audio['do_rms_norm'] = True
# config.audio['db_level'] = -27.0

# init speaker manager for multi-speaker training
# it maps speaker-id to speaker-name in the model and data-loader
# speaker_manager = SpeakerManager()
# speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
# config.model_args.num_speakers = speaker_manager.num_speakers

# language_manager = LanguageManager(config=config)
# config.model_args.num_languages = language_manager.num_languages

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
# tokenizer, config = TTSTokenizer.init_from_config(config)

# Init the model
model = Vits.init_from_config(config)
# model = Vits(config, ap, tokenizer, speaker_manager, language_manager)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512


In [11]:
# Init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(restore_path=RESTORE_PATH, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 6
 | > Num. of Torch Threads: 24
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/home/titor/Capstone/dubbing-ai/KhongKhunTTS/runs/KhongKhunTTS-TH-TSync2-February-03-2025_03+42PM-5977612
 > Restoring from best_model_YourTTS_VCTK.pth ...
 > Restoring Model...


 > `speakers.pth` is saved to /home/titor/Capstone/dubbing-ai/KhongKhunTTS/runs/KhongKhunTTS-TH-TSync2-February-03-2025_03+42PM-5977612/speakers.pth.
 > `speakers_file` is updated in the config.json.
 > `language_ids.json` is saved to /home/titor/Capstone/dubbing-ai/KhongKhunTTS/runs/KhongKhunTTS-TH-TSync2-February-03-2025_03+42PM-5977612/language_ids.json.
 > `language_ids_file` is updated in the config.json.


 > Partial model initialization...
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.0.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.0.weight_v
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.1.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.1.weight_v
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.2.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.2.weight_v
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.3.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.3.weight_v
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.4.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.4.weight_v
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.5.weight_g
 | > Lay

In [ ]:
trainer.fit()


 > EPOCH: 0/1000
 --> /home/titor/Capstone/dubbing-ai/KhongKhunTTS/runs/KhongKhunTTS-TH-TSync2-February-03-2025_03+42PM-5977612

 > TRAINING (2025-02-03 15:42:31) 
